## Loading-Data

In [36]:
# Importing modules
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
%matplotlib inline

In [37]:
df = pd.read_csv('AviationData.csv', encoding='ISO-8859-1')
df.head()

c:\Users\mutis\anaconda3\envs\learn-env\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (6,7,28) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
0,20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,UNK,Cruise,Probable Cause,NaN
1,20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,4.0,0.0,0.0,0.0,UNK,Unknown,Probable Cause,19-09-1996
2,20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,36.9222,-81.8781,NaN,NaN,...,Personal,NaN,3.0,NaN,NaN,NaN,IMC,Cruise,Probable Cause,26-02-2007
3,20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,12-09-2000
4,20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,1.0,2.0,NaN,0.0,VMC,Approach,Probable Cause,16-04-1980


In [38]:
# Getting more info about the dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88889 entries, 0 to 88888
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                88889 non-null  object 
 1   Investigation.Type      88889 non-null  object 
 2   Accident.Number         88889 non-null  object 
 3   Event.Date              88889 non-null  object 
 4   Location                88837 non-null  object 
 5   Country                 88663 non-null  object 
 6   Latitude                34382 non-null  object 
 7   Longitude               34373 non-null  object 
 8   Airport.Code            50249 non-null  object 
 9   Airport.Name            52790 non-null  object 
 10  Injury.Severity         87889 non-null  object 
 11  Aircraft.damage         85695 non-null  object 
 12  Aircraft.Category       32287 non-null  object 
 13  Registration.Number     87572 non-null  object 
 14  Make                    88826 non-null

We notice some missing values on various columns

### Data-Cleaning

#### Handling missing values

Checking the percentage of missing data

In [46]:
def missing_values():
    return df.isna().mean().sort_values(ascending=False)
missing_values()

FAR.Description           0.640267
Aircraft.Category         0.636787
Longitude                 0.613211
Latitude                  0.613109
Airport.Code              0.434392
Airport.Name              0.405860
Broad.phase.of.flight     0.305192
Publication.Date          0.155051
Engine.Type               0.078917
Report.Status             0.071359
Purpose.of.flight         0.069140
Number.of.Engines         0.067844
Weather.Condition         0.050002
Aircraft.damage           0.035730
Registration.Number       0.014362
Injury.Severity           0.011028
Country                   0.002534
Amateur.Built             0.001115
Location                  0.000586
Event.Date                0.000000
Accident.Number           0.000000
Investigation.Type        0.000000
Make                      0.000000
Model                     0.000000
Total.Fatal.Injuries      0.000000
Total.Serious.Injuries    0.000000
Total.Minor.Injuries      0.000000
Total.Uninjured           0.000000
Event.Id            

Both Schedule and Air.carrier have more than 80% missing values so we can just drop the columns

In [40]:
df.drop(['Schedule', 'Air.carrier'], axis=1, inplace=True)

In [41]:
df.columns

Index(['Event.Id', 'Investigation.Type', 'Accident.Number', 'Event.Date',
       'Location', 'Country', 'Latitude', 'Longitude', 'Airport.Code',
       'Airport.Name', 'Injury.Severity', 'Aircraft.damage',
       'Aircraft.Category', 'Registration.Number', 'Make', 'Model',
       'Amateur.Built', 'Number.of.Engines', 'Engine.Type', 'FAR.Description',
       'Purpose.of.flight', 'Total.Fatal.Injuries', 'Total.Serious.Injuries',
       'Total.Minor.Injuries', 'Total.Uninjured', 'Weather.Condition',
       'Broad.phase.of.flight', 'Report.Status', 'Publication.Date'],
      dtype='object')

Checking the make and model which are one of our main columns

In [47]:
missing_values()[['Make', 'Model']]

Make     0.0
Model    0.0
dtype: float64

Only a small percentage is missing so we drop the few rows with missing entries

In [43]:
df.dropna(subset=['Make', 'Model'], inplace=True)

In [44]:
df[['Make', 'Model']].isna().sum()

Make     0
Model    0
dtype: int64

Handling missing values for the columns with continuous data by imputing the mean

In [45]:
# columns with continuous data
injuries_info_cols = [
    'Total.Fatal.Injuries',
    'Total.Serious.Injuries',
    'Total.Minor.Injuries',
    'Total.Uninjured'
]

for col in injuries_info_cols:
    mean = np.floor(df[col].mean())
    df[col].fillna(mean, inplace=True)
    
# confirm no missing values
df[injuries_info_cols].isna().sum()

Total.Fatal.Injuries      0
Total.Serious.Injuries    0
Total.Minor.Injuries      0
Total.Uninjured           0
dtype: int64

Now we check other columns with categorical data